In [1]:
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim,os
import tokenize
import numpy as np
import pandas as pd

In [39]:
df_train = pd.read_csv('../../Data/train_clean.csv',encoding='utf-8')
qid_stack = df_train['qid1'].append(df_train['qid2'])
df_train_naExc = df_train[~df_train.isnull().any(axis=1)]
df_buf1 = df_train_naExc[['qid1','question1']]
df_buf2 = df_train_naExc[['qid2','question2']]
df_buf1.columns = ['qid','question']
df_buf2.columns = ['qid','question']
df_stack = df_buf1.append(df_buf2)
df_stack_dupExc = df_stack.drop_duplicates()

In [50]:
test_buf = df_stack_dupExc['question']#.head(1000)#.iloc[5000:7000]#head(1000)

In [51]:
len(df_stack_dupExc)

537931

In [52]:
test_LDA_buf = test_buf.tolist()
doc_set = test_LDA_buf

In [53]:
# compile sample documents into a list
# doc_buf = df_stack_dupExc['question']
# doc_buf = doc_buf.tolist()
# doc_set = doc_buf

In [54]:
#########
######### Tokenize the doc
#########
tokenizer = RegexpTokenizer(r'\w+')
# create English stop words list
en_stop = get_stop_words('en')

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()
###
def tokenize_doc(new_corpus):
    new_corpus
    texts = []
    ###
    ### Progressing bar setting
    totall_size = len(new_corpus)
    counter = 0
    progress = 0
    # loop through document list
    for i in new_corpus:    
        ### Monitor the progress
        if progress%(1000) ==0:
            bar = "[" + int(progress/1000)*"|"+">"+ (10-int(progress/1000))*"-" + str((progress/100)) + "%" "]"
            print(bar)
        #### count
        counter += 1
        progress = int(10000*(counter/totall_size))

        # clean and tokenize document string
        raw = i.lower()
        tokens = tokenizer.tokenize(raw)

        # remove stop words from tokens
        stopped_tokens = [i for i in tokens if not i in en_stop]
        
        # stem tokens
#         stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
        stemmed_tokens = []
        for i in stopped_tokens:
            try:
                stemmed_tokens = stemmed_tokens + [p_stemmer.stem(i)]
            except:
                print(i)
        # add tokens to list            
        texts.append(stemmed_tokens)
    return texts

######
######  predict the topic
######
def topic_predict(new_corpus, model, dictionary):
    tokenize_buf = tokenize_doc([new_corpus])
    doc_bow = [dictionary.doc2bow(text) for text in tokenize_buf]
    que_vec = [item for itemList in doc_bow for item in itemList]
    topic_vec = ldamodel[que_vec]

    word_count_array = np.empty((len(topic_vec), 2), dtype = np.object)
    for i in range(len(topic_vec)):
        word_count_array[i, 0] = topic_vec[i][0]
        word_count_array[i, 1] = topic_vec[i][1]

    idx = np.argsort(word_count_array[:, 1])
    idx = idx[::-1]
    word_count_array = word_count_array[idx]

    final = []
    final = ldamodel.print_topic(word_count_array[0, 0], len(word_count_array))#1)

    question_topic = final.split('*') ## as format is like "probability * topic"
    return question_topic
    #return question_topic[0]

In [55]:
##### Some word can not be  [stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]]
##### Such like "aed"
# new_corpus = doc_set
# texts = []
# # loop through document list
# for i in new_corpus:    
#     # clean and tokenize document string
#     raw = i.lower()
#     tokens = tokenizer.tokenize(raw)

#     # remove stop words from tokens
#     stopped_tokens = [i for i in tokens if not i in en_stop]
    
#     # stem tokens
#     try:
#         stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
#     except:
#         print(i)
#     # add tokens to list
#     texts.append(stemmed_tokens)


In [56]:
########## pre-processing ##########
####################################
    
# list for tokenized documents in loop
# texts = []

# # loop through document list
# for i in doc_set:
    
#     # clean and tokenize document string
#     raw = i.lower()
#     tokens = tokenizer.tokenize(raw)

#     # remove stop words from tokens
#     stopped_tokens = [i for i in tokens if not i in en_stop]
    
#     # stem tokens
#     stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    
#     # add tokens to list
#     texts.append(stemmed_tokens)
texts = tokenize_doc(doc_set)

[>----------0.0%]
[>----------0.0%]
[>----------0.0%]
[>----------0.0%]
[>----------0.0%]
[>----------0.0%]
[>----------0.0%]
[>----------0.0%]
[>----------0.0%]
[>----------0.0%]
[>----------0.0%]
[>----------0.0%]
[>----------0.0%]
[>----------0.0%]
[>----------0.0%]
[>----------0.0%]
[>----------0.0%]
[>----------0.0%]
[>----------0.0%]
[>----------0.0%]
[>----------0.0%]
[>----------0.0%]
[>----------0.0%]
[>----------0.0%]
[>----------0.0%]
[>----------0.0%]
[>----------0.0%]
[>----------0.0%]
[>----------0.0%]
[>----------0.0%]
[>----------0.0%]
[>----------0.0%]
[>----------0.0%]
[>----------0.0%]
[>----------0.0%]
[>----------0.0%]
[>----------0.0%]
[>----------0.0%]
[>----------0.0%]
[>----------0.0%]
[>----------0.0%]
[>----------0.0%]
[>----------0.0%]
[>----------0.0%]
[>----------0.0%]
[>----------0.0%]
[>----------0.0%]
[>----------0.0%]
[>----------0.0%]
[>----------0.0%]
[>----------0.0%]
[>----------0.0%]
[>----------0.0%]
[>----------0.0%]
aed
aed
[|>---------10.0%]
[

In [57]:
# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)

# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]

# generate LDA model
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=10, id2word = dictionary, passes=20)

In [58]:
dict_buf = {"tokenize_data": texts, 
            "dictionary": dictionary, 
            "encoding_corpus": corpus, 
            "lda_model":ldamodel}

In [59]:
import pickle

# selfref_list = [1, 2, 3]
# selfref_list.append(selfref_list)

output = open('ldamodel_traing_data.pkl', 'wb')

# Pickle dictionary using protocol 0.
pickle.dump(dict_buf, output)

# Pickle the list using the highest protocol available.
#pickle.dump(selfref_list, output, -1)

output.close()

In [36]:
# import pprint, pickle

# pkl_file = open('ldamodel_traing_data.pkl', 'rb')

# data2 = pickle.load(pkl_file)
# # pprint.pprint(data2)

# # data2 = pickle.load(pkl_file)
# # pprint.pprint(data2)

# pkl_file.close()

In [ ]:
print(ldamodel.print_topics(num_topics=10, num_words=4))

In [51]:
pivot = 1000
print(df_stack_dupExc['question'].iloc[pivot])
print(topic_predict(df_stack_dupExc['question'].iloc[pivot], ldamodel))

What is the best age to teach a child how to swim 
['0.039', '"differ" + 0.034', '"s" + 0.018', '"feel" + 0.015', '"human" + 0.013', '"like" + 0.010', '"10" + 0.010', '"type" + 0.009', '"one" + 0.008', '"anim" + 0.008', '"real"']


In [22]:
test_LDA_buf[10]

'Method to find separation of slits using fresnel biprism '

In [23]:
topic_predict(test_LDA_buf[10], ldamodel, dictionary)

['0.032',
 '"best" + 0.013',
 '"india" + 0.010',
 '"s" + 0.008',
 '"can" + 0.007',
 '"get" + 0.007',
 '"chang" + 0.006',
 '"countri" + 0.006',
 '"movi" + 0.004',
 '"porn" + 0.004',
 '"us"']